### 3. Clasificación de series de tiempo

- Elegir **una de las siguientes opciones**:
  - Transformar el problema de regresión abordado previamente en un problema de clasificación (por ejemplo, clasificar tendencias como "sube", "baja" o "estable").
  - Seleccionar **una nueva base de datos** específicamente orientada a clasificación de series de tiempo.

- Implementar las siguientes estructuras de modelos que permitan resolver el problema de clasificación:
  - MLP para clasificación
  - CNN para clasificación
  - LSTM para clasificación
  - CNN-LSTM para clasificación
  - Algoritmos clásicos de Machine Learning (SVM, Random Forest, etc.)

https://iris.who.int/bitstream/handle/10665/345329/9789240034228-eng.pdf?

In [1]:
import pandas as pd
import seaborn as sns
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.layers import Conv1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import optuna
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_class_weight


2025-05-12 16:42:15.342607: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-12 16:42:15.343207: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-12 16:42:15.345872: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-12 16:42:15.352572: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747089735.364675  274834 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747089735.36

In [2]:
df = pd.read_csv("Data/air_quality_clean.csv", parse_dates=['Datetime'])
df.set_index('Datetime', inplace=True)
df

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
Datetime,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,11.9,1046.0,1056.0,1692.0,1268.0,13.6,48.9,0.7578
2004-03-10 19:00:00,2.0,1292.0,9.4,955.0,1174.0,1559.0,972.0,13.3,47.7,0.7255
2004-03-10 20:00:00,2.2,1402.0,9.0,939.0,1140.0,1555.0,1074.0,11.9,54.0,0.7502
2004-03-10 21:00:00,2.2,1376.0,9.2,948.0,1092.0,1584.0,1203.0,11.0,60.0,0.7867
2004-03-10 22:00:00,1.6,1272.0,6.5,836.0,1205.0,1490.0,1110.0,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...
2005-04-04 10:00:00,3.1,1314.0,13.5,1101.0,539.0,1374.0,1729.0,21.9,29.3,0.7568
2005-04-04 11:00:00,2.4,1163.0,11.4,1027.0,604.0,1264.0,1269.0,24.3,23.7,0.7119
2005-04-04 12:00:00,2.4,1142.0,12.4,1063.0,603.0,1241.0,1092.0,26.9,18.3,0.6406


En este caso vamos a hacer lo de bajo, medio y alto nivel de monóxido de cárbono en el aire. Para esto investigaramos cuales suelen ser los niveles en el aire para poderlos clasificar. 
Según la OMS (Organización Mundial de la Salud) y otras agencias como la EPA (USA):

| Nivel de CO  | Rango aproximado (mg/m³) |
| ------------ | ------------------------ |
| **Bajo**     | 0 – 4.4                  |
| **Moderado** | 4.5 – 9.0 >              |



In [3]:
def clasificar_binario(valores):
    clases = []
    for val in valores:
        if val <= 4.4:
            clases.append(0)  # Normal
        else:
            clases.append(1)  # No normal (Moderado o Alto)
    return np.array(clases)

In [4]:
# 0 es baja
# 1 es no normal, ya sea moderado o alto

In [5]:
y_bin = clasificar_binario(df["CO(GT)"].values)


In [6]:
unique, counts = np.unique(y_bin, return_counts=True)

conteo_df = pd.DataFrame({'Clase': unique, 'Cantidad': counts})
conteo_df['Etiqueta'] = conteo_df['Clase'].map({0: 'Normal', 1: 'No normal'})

print(conteo_df)


   Clase  Cantidad   Etiqueta
0      0      8772     Normal
1      1       585  No normal


Habiamos intententado que las clases fueran, bajo, moderado y alto. Pero como de alto solo habia 12 valores, lo integramos a una que se llamara "no Normal"

# MLP

In [7]:
y_bin = clasificar_binario(df["CO(GT)"].values)
X = df.drop(columns=["CO(GT)"]).values # todas menos la variable a predecir

In [8]:
# Escalar
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# Crear ventanas de tiempo aplanadas 
def crear_ventanas_bin(X, y, window_size):
    Xs, ys = [], []
    for i in range(window_size, len(y)):
        Xs.append(X[i-window_size:i].flatten())
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

window_size = 12
X_seq, y_seq = crear_ventanas_bin(X_scaled, y_bin, window_size)


In [10]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

In [11]:
# Modelo MLP 
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # salida binaria

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# EarlyStopping 
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamiento
model.fit(X_train, y_train,
          validation_split=0.2,
          epochs=50,
          batch_size=32,
          callbacks=[early_stop],
          verbose=1)


Epoch 1/50


/home/maripau/Documents/ITESO/Semestre6/Modelos no lineales/ProyectoFinal/air_proyecto/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-12 16:42:16.910969: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8974 - loss: 0.2702 - val_accuracy: 0.8817 - val_loss: 0.3243
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9573 - loss: 0.1189 - val_accuracy: 0.8957 - val_loss: 0.3232
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9593 - loss: 0.1065 - val_accuracy: 0.8991 - val_loss: 0.3390
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9559 - loss: 0.1085 - val_accuracy: 0.9011 - val_loss: 0.3071
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9624 - loss: 0.0992 - val_accuracy: 0.9051 - val_loss: 0.3001
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9631 - loss: 0.0965 - val_accuracy: 0.9017 - val_loss: 0.2984
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9595 - loss: 0.0972 - val_accuracy: 0.9078 - val_loss: 0.3013
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9590 - loss: 0.1064 - val_accuracy: 0.8977 - val_

In [12]:
# Evaluación
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step


In [13]:
print("Reporte de clasificación binaria:")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación binaria:
              precision    recall  f1-score   support

      Normal       0.97      0.97      0.97      1756
   No normal       0.56      0.54      0.55       113

    accuracy                           0.95      1869
   macro avg       0.77      0.76      0.76      1869
weighted avg       0.95      0.95      0.95      1869

Exactitud total: 0.947


* El modelo detecta muy bien los casos normales, con precisión y recall cercanos al 98%.
* En los casos "No normales" (niveles elevados de CO), el desempeño baja:

Precisión de 60% → cuando predice “no normal”, acierta el 60% de las veces.

Recall de 54% → detecta poco más de la mitad de los casos reales de “no normal”.

Esto es normal ya que las clases estaban desbalanceadas.

El modelo es muy confiable para identificar niveles normales de CO(GT), pero tiene dificultades para detectar eventos poco frecuentes de contaminación elevada. Aun así, puede servir como un buen sistema de alerta preliminar, especialmente si se complementa con técnicas de balanceo de clases o ajuste de pesos.

# CNN clasificación

In [14]:
# Crear ventanas multivariadas para CNN 
def crear_ventanas_cnn(X, y, window_size):
    Xs, ys = [], []
    for i in range(window_size, len(y)):
        Xs.append(X[i-window_size:i, :])  
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

window_size = 24
X_seq, y_seq = crear_ventanas_cnn(X_scaled, y_bin, window_size)


In [ ]:
# Modelo CNN 
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid')) #por que son dos categorias

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/home/maripau/Documents/ITESO/Semestre6/Modelos no lineales/ProyectoFinal/air_proyecto/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# EarlyStopping 
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar
model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 108), dtype=float32). Expected shape (None, 108, 108), but input has incompatible shape (None, 108)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 108), dtype=float32)
  • training=True
  • mask=None

In [ ]:
# Evaluar 
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


Si la probabilidad es mayor a 0.5, se predice la clase 1 (No normal).

Si es menor o igual a 0.5, se predice la clase 0 (Normal).

In [ ]:
print("Reporte de clasificación CNN:")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación CNN:
              precision    recall  f1-score   support

      Normal       0.96      1.00      0.98      1754
   No normal       0.82      0.27      0.41       113

    accuracy                           0.95      1867
   macro avg       0.89      0.64      0.69      1867
weighted avg       0.95      0.95      0.94      1867

Exactitud total: 0.952


Ambos modelos clasifican muy bien la clase “Normal”, como es de esperarse por el desbalance de las clases.

El CNN tiene mayor precisión en detectar "No normal", pero mucho menor recall (predice pocos positivos, pero con menos error).

El MLP tiene mejor balance entre precisión y recall, por lo tanto mejor F1-score para la clase minoritaria.

La exactitud total es prácticamente la misma en ambos casos, pero no refleja el verdadero rendimiento sobre la clase difícil ("No normal").

Si nos importara  no pasar por alto ningún caso de monóxido elevado, el MLP es mejor (más recall).

Si preferimos menos falsos positivos (es decir, cuando diga “No normal” esté bien seguro), el CNN puede servir mejor.

# LSTM

In [ ]:
# Crear ventanas multivariadas para LSTM 
def crear_ventanas_lstm(X, y, window_size):
    Xs, ys = [], []
    for i in range(window_size, len(y)):
        Xs.append(X[i-window_size:i, :])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

window_size = 24
X_seq, y_seq = crear_ventanas_lstm(X_scaled, y_bin, window_size)

In [ ]:
# Modelo LSTM 
model = Sequential()
model.add(LSTM(64, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # sigmoide por ser binaria

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/home/maripau/Documents/ITESO/Semestre6/Modelos no lineales/ProyectoFinal/air_proyecto/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# EarlyStopping 
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamiento
model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9569 - loss: 0.2613 - val_accuracy: 0.8628 - val_loss: 0.4489
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9578 - loss: 0.1651 - val_accuracy: 0.8628 - val_loss: 0.3741
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9518 - loss: 0.1633 - val_accuracy: 0.8628 - val_loss: 0.3866
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9577 - loss: 0.1291 - val_accuracy: 0.8675 - val_loss: 0.3425
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9587 - loss: 0.1090 - val_accuracy: 0.8929 - val_loss: 0.2845
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9594 - loss: 0.1017 - val_accuracy: 0.8829 - val_loss: 0.3253
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9594 - loss: 0.0989 - val_accuracy: 0.8896 - val_loss: 0.3176
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9637 - loss: 0.0901 - val_accuracy: 0.

In [ ]:
# Evaluar
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [ ]:
print("Reporte de clasificación LSTM:")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación LSTM:
              precision    recall  f1-score   support

      Normal       0.97      0.98      0.97      1754
   No normal       0.62      0.50      0.55       113

    accuracy                           0.95      1867
   macro avg       0.80      0.74      0.76      1867
weighted avg       0.95      0.95      0.95      1867

Exactitud total: 0.951


El modelo LSTM clasifica muy bien la clase "Normal", como todos los anteriores.

En la clase "No normal", tiene un buen equilibrio entre precisión (62%) y recall (50%), lo que resulta en un F1-score razonable (0.55).

Mejora el recall respecto al CNN (que fue solo del 27%) y se acerca al MLP.

El LSTM mejora el balance entre sensibilidad (recall) y precisión para los casos "No normales".

Tiene una predicción más justa y estable que el CNN.

Si nuestro objetivo fuera detectar niveles elevados de monóxido sin perder demasiados casos, este modelo es muy competitivo, al nivel del MLP.

# CNN-LSTM

In [ ]:
# Crear ventanas CNN-LSTM (forma 4D: muestras, subseq, pasos, features) ---
def crear_ventanas_cnn_lstm(X, y, window_size, subseq_len):
    Xs, ys = [], []
    for i in range(window_size, len(y)):
        full_seq = X[i - window_size:i, :] 
        subseqs = full_seq.reshape((window_size // subseq_len, subseq_len, X.shape[1]))
        Xs.append(subseqs)
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

window_size = 24
subseq_len = 4  # 24 pasos → 6 subsecuencias de 4 pasos

X_seq, y_seq = crear_ventanas_cnn_lstm(X_scaled, y_bin, window_size, subseq_len)


In [ ]:
# Modelo CNN-LSTM 
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=2, activation='relu'),
                          input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64, activation='tanh'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Salida binaria

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/home/maripau/Documents/ITESO/Semestre6/Modelos no lineales/ProyectoFinal/air_proyecto/lib/python3.12/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# EarlyStopping 
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamiento
model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9152 - loss: 0.2934 - val_accuracy: 0.8628 - val_loss: 0.3979
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9560 - loss: 0.1675 - val_accuracy: 0.8628 - val_loss: 0.4326
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9556 - loss: 0.1656 - val_accuracy: 0.8628 - val_loss: 0.3804
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9566 - loss: 0.1596 - val_accuracy: 0.8628 - val_loss: 0.3764
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9519 - loss: 0.1576 - val_accuracy: 0.8628 - val_loss: 0.3513
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9578 - loss: 0.1426 - val_accuracy: 0.8628 - val_loss: 0.3578
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9594 - loss: 0.1332 - val_accuracy: 0.8628 - val_loss: 0.3584
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9558 - loss: 0.1355 - val_accuracy: 0.

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [ ]:
print("Reporte de clasificación CNN-LSTM:")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación CNN-LSTM:
              precision    recall  f1-score   support

      Normal       0.95      0.99      0.97      1754
   No normal       0.58      0.25      0.35       113

    accuracy                           0.94      1867
   macro avg       0.77      0.62      0.66      1867
weighted avg       0.93      0.94      0.93      1867

Exactitud total: 0.944


El modelo clasifica con alta precisión los casos "Normales", lo cual es esperado por ser la clase mayoritaria.

En cambio, para los casos "No normales" (niveles elevados de CO):

Solo detecta correctamente 25% de ellos (bajo recall).

Cuando predice “No normal”, acierta el 58% de las veces (precisión aceptable).

El F1-score es bajo (0.35), lo que refleja la dificultad para capturar esta clase minoritaria.

# Random Forest

In [ ]:
# modelo Random Forest ---
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
# Predicción
y_pred = rf.predict(X_test)

In [ ]:
print("Reporte de clasificación — Random Forest:")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación — Random Forest:
              precision    recall  f1-score   support

      Normal       0.98      0.97      0.97      1759
   No normal       0.59      0.63      0.61       113

    accuracy                           0.95      1872
   macro avg       0.78      0.80      0.79      1872
weighted avg       0.95      0.95      0.95      1872

Exactitud total: 0.951


El modelo tiene muy alto desempeño para la clase Normal (como los otros modelos).

Para la clase No normal:

Recall: 0.63 el mejor entre todos los modelos, hasta ahora

Precisión: 0.59 bastante equilibrado.

F1-score: 0.61 mejor resultado hasta ahora para esta clase minoritaria.



El Random Forest supera a todos los modelos anteriores en su capacidad para detectar casos "No normales". Tiene el mejor recall y F1-score para esta clase crítica, manteniendo una exactitud general muy alta. Es, por tanto, una de las mejores opciones si tu objetivo es detectar contaminación por CO(GT).

##### ¿Por qué Random Forest tuvo mejores resultados?
Maneja bien desequilibrios de clase:
Random Forest es robusto ante desequilibrios de clases. Al construir múltiples árboles sobre subconjuntos aleatorios, algunos árboles ven más casos de la clase minoritaria, lo que mejora su sensibilidad (recall) en esa clase.

 Modelo no secuencial: 
A diferencia de LSTM o CNN-LSTM, que dependen del orden temporal y pueden diluir señales débiles de eventos raros, el Random Forest ve cada instante como un punto independiente y se enfoca en la relación entre variables en ese momento, lo cual puede capturar mejor señales puntuales de contaminación.



# XGBOOST

Ahora decidimos hacer un xgboost por que el xgboost inicia con un árbol pequeño y va corrigiendo los errores hasta mejorar poco a poco. 

In [ ]:
# --- Entrenar modelo XGBoost ---
xgb = XGBClassifier(
    n_estimators=100, #número de árboles
    max_depth=10, #profundidad de cada árbol
    learning_rate=0.1,
    scale_pos_weight=(sum(y_train == 0) / sum(y_train == 1)),  # balancear clases
    use_label_encoder=False, # no advertencias de xgboost
    eval_metric='logloss', #log loss para binario
    random_state=42
)

xgb.fit(X_train, y_train)

/home/maripau/Documents/ITESO/Semestre6/Modelos no lineales/ProyectoFinal/air_proyecto/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:14:53] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
y_pred = xgb.predict(X_test)

print("Reporte de clasificación — XGBoost:")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación — XGBoost:
              precision    recall  f1-score   support

      Normal       0.98      0.92      0.95      1759
   No normal       0.39      0.78      0.52       113

    accuracy                           0.91      1872
   macro avg       0.69      0.85      0.73      1872
weighted avg       0.95      0.91      0.93      1872

Exactitud total: 0.912


Clase "Normal" (0):
Muy alta precisión (0.98): casi nunca predice “Normal” cuando no lo es.

Muy buen recall (0.92): detecta la gran mayoría de los casos normales.

F1-score alto: equilibrio perfecto entre precisión y sensibilidad.

Clase "No normal" (1):
Precisión moderada (0.39): se equivoca a veces cuando dice “No normal”.

Recall excelente (0.78): detecta casi 8 de cada 10 episodios reales de contaminación. No esta nada mal

F1-score de 0.52: mejor que cualquier otro modelo anterior en esta clase.

Aunque la exactitud total es más baja (91.2%), el modelo XGBoost es el mejor en detectar episodios "No normales", que son los más importantes y difíciles de predecir. Su alto recall (0.78) lo convierte en una herramienta excelente para alertas tempranas, aunque a costa de más falsos positivos.

# Conclusión

El propósito fue transformar el problema de regresión de predicción de CO(GT) en una tarea de clasificación binaria, donde la clase "Normal" incluye niveles de CO menores o iguales a 4.4 mg/m³ (según la guía de calidad del aire de la OMS), y la clase "No normal" agrupa los valores por encima de ese umbral. La meta principal fue evaluar qué modelos detectan mejor los casos "No normales", que representan condiciones potencialmente peligrosas para la salud.

#### Modelos realizados

| Modelo        | Accuracy | Recall (No normal) | F1-score (No normal) |
| ------------- | -------- | ------------------ | -------------------- |
| MLP           | 0.950    | 0.54               | 0.57                 |
| CNN           | 0.952    | 0.27               | 0.41                 |
| LSTM          | 0.951    | 0.50               | 0.55                 |
| CNN-LSTM      | 0.944    | 0.25               | 0.35                 |
| Random Forest | 0.951    | 0.63               | 0.61                 |
| XGBoost       | 0.912    | **0.78**           | **0.52**             |


- Todos los modelos neuronales tuvieron alta precisión general, especialmente en la clase "Normal", pero varios presentaron bajo recall en la clase "No normal", lo que indica que fallan en detectar eventos de contaminación.
- El modelo XGBoost, aunque tuvo menor exactitud general, alcanzó el mejor recall (0.78) para la clase "No normal". Esto lo convierte en el más útil para sistemas de alerta temprana, donde detectar correctamente niveles elevados es más importante que equivocarse ocasionalmente.
La arquitectura y los hiperparámetros del XGBoost se ajustaron para manejar el desbalance de clases usando el parámetro scale_pos_weight. Puede explicar por que el mojoró el resultado




# Modelos neuronles con oversampling para el desbalance.

In [ ]:
#balance de las clases
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
print("Pesos de clase:", class_weights)


Pesos de clase: {0: np.float64(0.533743208464398), 1: np.float64(7.908898305084746)}


In [ ]:
# Modelo MLP balanceado
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Salida binaria

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/home/maripau/Documents/ITESO/Semestre6/Modelos no lineales/ProyectoFinal/air_proyecto/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Early stopping 
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamiento con pesos de clase 
model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    class_weight=class_weights,
    verbose=1
)


Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7501 - loss: 0.4834 - val_accuracy: 0.8681 - val_loss: 0.3473
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8654 - loss: 0.2956 - val_accuracy: 0.7296 - val_loss: 0.5127
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8935 - loss: 0.2388 - val_accuracy: 0.6627 - val_loss: 0.6419
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8577 - loss: 0.2662 - val_accuracy: 0.7938 - val_loss: 0.4464
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8711 - loss: 0.2463 - val_accuracy: 0.8594 - val_loss: 0.3240
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8855 - loss: 0.2378 - val_accuracy: 0.7189 - val_loss: 0.5754
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8532 - loss: 0.2447 - val_accuracy: 0.7490 - val_loss: 0.4984
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8826 - loss: 0.2252 - val_accuracy: 0.

In [ ]:
# Evaluación
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step


In [ ]:
print("Reporte de clasificación MLP (balanceado):")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación MLP (balanceado):
              precision    recall  f1-score   support

      Normal       0.99      0.87      0.92      1754
   No normal       0.28      0.83      0.42       113

    accuracy                           0.86      1867
   macro avg       0.64      0.85      0.67      1867
weighted avg       0.95      0.86      0.89      1867

Exactitud total: 0.863


### Comparación mlp sin balanceo y con
| Métrica               | MLP original | MLP balanceado |
| --------------------- | ------------ | -------------- |
| Accuracy              | 0.950        | 0.863          |
| Recall (No normal)    | 0.54         | **0.83** mucho mejor    |
| F1 (No normal)        | 0.57         | **0.42** ↓     |
| Precisión (No normal) | 0.53         | **0.28** ↓     |



El modelo balanceado sacrificó exactitud general, pero a cambio logró detectar correctamente la gran mayoría de los casos "No normal" (83% de recall, el más alto de todos tus MLPs).

La precisión bajó bastante (0.28), lo que significa que ahora el modelo lanza más falsos positivos (predice "No normal" cuando no lo es).

El MLP con class_weight mejoró muchísimo la sensibilidad al detectar niveles peligrosos de CO (83% de recall), lo que es clave para sistemas de alerta. Aunque pierde precisión, es preferible si nuestra prioridad es no dejar pasar ningún caso “No normal”. Como todo tiene sus pros y sus contras. 



# CNN Balanceada

In [ ]:
# Calcular pesos de clase 
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
print("Pesos de clase:", class_weights)

Pesos de clase: {0: np.float64(0.533743208464398), 1: np.float64(7.908898305084746)}


In [ ]:
# Modelo CNN 
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Clasificación binaria

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/home/maripau/Documents/ITESO/Semestre6/Modelos no lineales/ProyectoFinal/air_proyecto/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# EarlyStopping 
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamiento 
model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    class_weight=class_weights,
    verbose=1
)

Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8178 - loss: 0.4989 - val_accuracy: 0.6037 - val_loss: 0.6694
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8569 - loss: 0.2915 - val_accuracy: 0.7664 - val_loss: 0.4817
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8759 - loss: 0.2509 - val_accuracy: 0.8179 - val_loss: 0.4054
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8920 - loss: 0.2430 - val_accuracy: 0.7744 - val_loss: 0.4949
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8870 - loss: 0.2222 - val_accuracy: 0.7497 - val_loss: 0.5111
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8755 - loss: 0.2164 - val_accuracy: 0.8434 - val_loss: 0.3489
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8996 - loss: 0.2102 - val_accuracy: 0.7865 - val_loss: 0.4499
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8992 - loss: 0.1856 - val_accuracy: 0.

In [ ]:
# Evaluación
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [ ]:
print("Reporte de clasificación CNN (balanceado):")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación CNN (balanceado):
              precision    recall  f1-score   support

      Normal       0.99      0.87      0.92      1754
   No normal       0.29      0.81      0.42       113

    accuracy                           0.87      1867
   macro avg       0.64      0.84      0.67      1867
weighted avg       0.94      0.87      0.89      1867

Exactitud total: 0.866



 
| Métrica               | CNN original | CNN balanceado |
| --------------------- | ------------ | -------------- |
| Accuracy              | 95.2%        | 86.6%          |
| Recall (No normal)    | 0.27         | **0.81** mejor   |
| Precisión (No normal) | 0.82         | **0.29** ↓     |
| F1-score (No normal)  | 0.41         | **0.42** ↔     |

El recall de la clase “No normal” pasó de 27% a 81%, una mejora enorme, lo que significa que ahora el modelo sí detecta la mayoría de los episodios de CO elevados.

La precisión bajó, como es natural cuando el modelo predice más casos positivos (hay más falsos positivos).

El F1-score se mantuvo, pero con un perfil distinto: más sensibilidad, menos precisión.

Accuracy bajó, porque se sacrifica algo de rendimiento en la clase mayoritaria ("Normal") para detectar mejor la minoritaria.

Al usar class_weight, la CNN se vuelve mucho más útil para detectar niveles peligrosos de CO, con un recall de 81% que la convierte en un modelo efectivo para aplicaciones de monitoreo o alerta. Aunque su precisión baja, esto es aceptable si tu objetivo es no dejar pasar casos de contaminación.


# LSTM

In [ ]:
# balanceo
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
print("Pesos de clase:", class_weights)

Pesos de clase: {0: np.float64(0.533743208464398), 1: np.float64(7.908898305084746)}


In [ ]:
# Modelo LSTM 
model = Sequential()
model.add(LSTM(64, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/home/maripau/Documents/ITESO/Semestre6/Modelos no lineales/ProyectoFinal/air_proyecto/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# EarlyStopping 
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamiento con pesos de clase 
model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    class_weight=class_weights,
    verbose=1
)

Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8671 - loss: 0.5687 - val_accuracy: 0.6305 - val_loss: 0.6306
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7764 - loss: 0.4897 - val_accuracy: 0.8253 - val_loss: 0.4395
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8526 - loss: 0.4210 - val_accuracy: 0.6185 - val_loss: 0.6652
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8746 - loss: 0.3108 - val_accuracy: 0.8715 - val_loss: 0.3344
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8891 - loss: 0.2614 - val_accuracy: 0.7965 - val_loss: 0.4485
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8971 - loss: 0.2347 - val_accuracy: 0.8788 - val_loss: 0.3366
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8969 - loss: 0.2215 - val_accuracy: 0.8293 - val_loss: 0.3939
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8903 - loss: 0.2241 - val_accuracy: 0.

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [ ]:
print("Reporte de clasificación LSTM (balanceado):")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación LSTM (balanceado):
              precision    recall  f1-score   support

      Normal       0.99      0.92      0.95      1754
   No normal       0.39      0.80      0.52       113

    accuracy                           0.91      1867
   macro avg       0.69      0.86      0.74      1867
weighted avg       0.95      0.91      0.93      1867

Exactitud total: 0.912


El modelo detecta correctamente 80% de los casos "No normal" → excelente mejora de recall.

Precisión de la clase minoritaria (0.39) es razonable, considerando el desbalance y que el modelo intenta no dejar pasar alertas.

El modelo sigue teniendo muy buen desempeño en la clase "Normal" (recall de 92% y F1 de 0.95).

Accuracy general de 91% lo pone al nivel del XGBoost, pero con la ventaja de usar la secuencia temporal.

Tu LSTM balanceado es uno de los mejores modelos para detectar condiciones de CO elevadas, logrando un excelente balance entre recall alto (80%) y exactitud total (91%), algo que los otros modelos neuronales no alcanzaban sin perder mucha precisión.
Además, aprovecha la estructura secuencial del problema, lo cual le da una ventaja conceptual sobre los modelos clásicos como XGBoost o RF.

# CNN_LSTM


In [ ]:
# balanceo
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
print("Pesos de clase:", class_weights)


In [ ]:
# --- Modelo CNN-LSTM ---
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=2, activation='relu'),
                          input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64, activation='tanh'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/home/maripau/Documents/ITESO/Semestre6/Modelos no lineales/ProyectoFinal/air_proyecto/lib/python3.12/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    class_weight=class_weights,
    verbose=1
)


Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8423 - loss: 0.5505 - val_accuracy: 0.2811 - val_loss: 0.8739
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6700 - loss: 0.5028 - val_accuracy: 0.4183 - val_loss: 0.7854
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7655 - loss: 0.4601 - val_accuracy: 0.3989 - val_loss: 0.8958
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7954 - loss: 0.4125 - val_accuracy: 0.5428 - val_loss: 0.8092
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8041 - loss: 0.3767 - val_accuracy: 0.6499 - val_loss: 0.6194
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8238 - loss: 0.3519 - val_accuracy: 0.7938 - val_loss: 0.4329
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8401 - loss: 0.3081 - val_accuracy: 0.8333 - val_loss: 0.3976
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8729 - loss: 0.2901 - val_accuracy: 0.

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [ ]:
print("Reporte de clasificación CNN-LSTM (balanceado):")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación CNN-LSTM (balanceado):
              precision    recall  f1-score   support

      Normal       0.98      0.88      0.93      1754
   No normal       0.26      0.65      0.38       113

    accuracy                           0.87      1867
   macro avg       0.62      0.77      0.65      1867
weighted avg       0.93      0.87      0.89      1867

Exactitud total: 0.868


El modelo sigue siendo muy fuerte para la clase "Normal", con una alta precisión (0.98) y buen recall (0.88).

Para la clase "No normal", el modelo logra un recall de 65%, lo que significa que detecta 2 de cada 3 episodios de contaminación (CO alto).

La precisión de 0.26 indica que hay bastantes falsos positivos, es decir, el modelo a veces predice "No normal" cuando no lo es.

El F1-score de 0.38 para la clase minoritaria muestra que hay un esfuerzo efectivo pero limitado para balancear detección vs falsos positivos.

El modelo CNN-LSTM mejora notablemente el recall respecto al CNN original, pero no logra superar al LSTM balanceado, que sigue siendo el mejor modelo en balance entre detección de eventos peligrosos y estabilidad.
Este modelo aún es útil si se busca un sistema más general que combine detección local (CNN) y secuencial (LSTM), pero se queda corto en precisión al detectar CO elevado.

# Conclusión final

Sin balanceo, los modelos neuronales mostraban alta exactitud general, pero muy bajo recall en la clase minoritaria (“No normal”), lo que significa que omitían la mayoría de los episodios de contaminación.

Al aplicar class_weight para dar mayor importancia a la clase minoritaria durante el entrenamiento:

Se logró un aumento significativo del recall para “No normal” (por ejemplo, de 27% a 81% en CNN) mucha diferencia.

Se observó una reducción aceptable en la precisión general, pero con un gran beneficio en la sensibilidad del modelo ante condiciones de riesgo.

El LSTM balanceado fue el modelo con mejor desempeño general: alto recall (80%) en la clase crítica, mejor F1-score para “No normal” (0.52) y una exactitud total sólida del 91.2%. Este modelo ofrece la mejor combinación entre detección efectiva de episodios contaminantes y estabilidad en las predicciones.
